In [1]:
from google.cloud import vision
import io
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
from PIL import Image

In [2]:
# Path to google private keys on personnal computers
KEY_GONXHE = 'C:/Users/gonxh/Documents/EPFL/Master/MA3/Foundations_of_DH/DH - Rolandi Libretti-a52be57f8f03.json'
KEY_ELISA = '/Users/elisamichelet/Documents/EPFLMaster/FDH/Projet/private_key.json'

In [3]:
def resize_image(file_without_extension):
    ''' Resizes the image to make it the same size as the segmentation output '''
    #Load npy
    image_path = "./data/Antigone/0_Images/" + file_without_extension + ".jpg"
    segmentation_path = "./data/Antigone/1_Segmentation_results/" + file_without_extension + ".npy"
    # Output data
    data = np.load(segmentation_path)
    # Resize original image (test)
    img = cv2.imread(image_path,0)
    img = cv2.resize(img, (np.shape(data)[2], np.shape(data)[1]))
    img = Image.fromarray(img)
    img.save(image_path)

In [4]:
def make_df(texts, filename):
    ''' Saves a dataframe with the text in the image and its coordinates '''
    df = pd.DataFrame()
    for text in texts[1:]:
        df_row = pd.DataFrame({"Text":[text.description], 
                    "Top_Left_X":[text.bounding_poly.vertices[0].x],
                    "Top_Left_Y":[text.bounding_poly.vertices[0].y],
                    "Bottom_Right_X":[text.bounding_poly.vertices[2].x],
                    "Bottom_Right_Y":[text.bounding_poly.vertices[2].y]}) 
        df = df.append(df_row, ignore_index=True)
    df.to_csv('./data/Antigone/2_OCR_results/annotations_' + filename + '.csv')  

In [5]:
def detect_text(filename, key_path):
    ''' Performs the OCR on one image '''
    
    # Google client
    client = vision.ImageAnnotatorClient.from_service_account_json(key_path)
    
    # Image to OCR
    with io.open("./data/Antigone/0_Images/" + filename + ".jpg", 'rb') as image_file:
        content = image_file.read()   
    # OCR
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    # Saving result in a dataframe
    make_df(texts, filename)
    


In [6]:
def OCR(key_path):
    ''' Performs the OCR on the entire libretto '''
    
    # Going through all the images
    for filename in os.listdir("./data/Antigone/0_Images"):
        if filename.endswith(".jpg"): 
            file_without_extension = os.path.splitext(filename)[0]
            # Resize image
            resize_image(file_without_extension)
            # OCR
            detect_text(file_without_extension, key_path)
            continue
        else:
            continue
    

In [7]:
OCR(KEY_ELISA)